In [1]:
# This is a notebook to do experiment requested by Nathan for checking token-wise length bias of rewards 
# (my guess: we'll see a lot at smaller scales, not so much at bigger)
import pandas as pd
from transformers import AutoTokenizer, pipeline, AutoModelForSequenceClassification
import torch
from datasets import load_dataset, Dataset
from tqdm import tqdm
import pickle

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-12-21 14:16:15,009] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
from transformers import PreTrainedModel, LlamaConfig, LlamaModel, LlamaTokenizer
import torch.nn as nn
import torch
from typing import Optional, List

# UltraRM format so that we don't need to re-run a ton of times for each unique input
class LlamaRewardModel(PreTrainedModel):
    config_class = LlamaConfig
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.regression_head = nn.Linear(self.config.hidden_size, 1, bias=False)

    def forward( # args are the same as LlamaForCausalLM
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        transformer_outputs = self.model(
                                input_ids,
                                attention_mask=attention_mask,
                                position_ids=position_ids,
                                past_key_values=past_key_values,
                                inputs_embeds=inputs_embeds,                               
                            )

        hidden_states = transformer_outputs[0]
        rewards = self.regression_head(hidden_states).squeeze(-1)
        
        return rewards

In [26]:
def hhrlhf_preproc(inp, out):
    resp = inp
    if "###Human: " not in inp: 
        resp = "###Human: "+inp
    if "###Assistant: " not in out: 
        resp = resp+" ###Assistant: "
    resp = resp+out
    return resp
    
def progress_causalrm(inps, mod, tok, cache):
    cache.clear()
    inps.reverse()
    scores = []
    # TODO test to see if batching is needed, currently this assumes no batching
    for i in tqdm(range(len(inps))):
        #print(inps[i])
        # switch batck to original HHRLHF format (TODO this is weird)
        question, answer = inps[i]
        question = question.replace("###Assistant:", "\n\nAssistant:").replace("###Human:", "\n\nHuman:")
        question = (question + "\n\nAssistant:").strip()
        tokrewards = None
        # used cached full response if we can
        for k in cache.keys():
            if question in k: 
                tokrewards = cache[question]
        modinps = tok(question+answer, return_tensors='pt')
        
        if tokrewards==None: 
            modinps = modinps.to(mod.device)
            # first one in the cache
            cache[question] = mod(**modinps).tolist()[0]
            tokrewards = cache[question]
        if i==0:
            print(question+answer)
            #print(len(modinps))
            #print(len(tokrewards))
            
        # get the token score based on length of stuff
        scores.append(tokrewards[len(modinps.input_ids[0])-1])   
        #print(len(tokrewards))
        #print(len(modinps.input_ids[0])-1)
    scores.reverse()
    return scores

def progress_oasst(inps, mod, tok):
    scores = []
    # TODO test to see if batching is needed
    for i in tqdm(range(len(inps))):
        #print(inps[i])
        # switch batck to original HHRLHF format (TODO this is weird)
        question, answer = inps[i]
        question = question.replace("###Assistant:", "\n\nAssistant:").replace("###Human:", "Human:")
        # question = (question + "\n\nAssistant:").strip()
        question = question.strip()+"\n\nAssistant: "
        if i==0:
            print(question, answer)
        modinps = tokenizer(question, answer, return_tensors='pt').to(mod.device)
        scores.append(mod(**modinps).logits[0].cpu().detach())
    return scores

def load_rm(rmname): 
    # TODO maybe offload input formatting into here as well
    if rmname=="weqweasdas/hh_rlhf_rm_open_llama_3b":
        rm_tokenizer = AutoTokenizer.from_pretrained("weqweasdas/hh_rlhf_rm_open_llama_3b")
        rm_pipe = pipeline(
          "sentiment-analysis",
          model="weqweasdas/hh_rlhf_rm_open_llama_3b",
          device=1,
          tokenizer=rm_tokenizer,
          model_kwargs={"torch_dtype": torch.bfloat16}
        )
        pipe_kwargs = {
          "return_all_scores": True,
          "function_to_apply": "none",
          "batch_size": 4
        }
        def progress_pipe(inps, kwargs, pipe): 
            chunk=16
            results = []
            print(inps[0])
            for i in tqdm(range(0, len(inps), chunk)):
                results.extend(pipe(inps, **kwargs))
            return results
        return lambda texts: [output[0]["score"] for output in progress_pipe([hhrlhf_preproc(i, o) for i, o in texts], pipe_kwargs, rm_pipe)]
    if rmname=="OpenAssistant/reward-model-deberta-v3-large-v2":
        rank_model, toker = AutoModelForSequenceClassification.from_pretrained(rmname, device_map=1), AutoTokenizer.from_pretrained(rmname)
        return lambda texts: progress_oasst(texts, rank_model, toker)
    if rmname=="openbmb/UltraRM-13b":
        # load in models
        rtoker = LlamaTokenizer.from_pretrained("openbmb/UltraRM-13b")
        rmodel = LlamaRewardModel.from_pretrained("openbmb/UltraRM-13b", device_map=1, torch_dtype=torch.bfloat16)
        rmodel.eval()
        # cache full rewards (tokenwise) for the longest seqeunce (reverse the passing order), then we can pull from that 
        reward_cache = {}
        
        return lambda texts: progress_causalrm(texts, rmodel, rtoker, reward_cache)

def get_token_inps(inputs, toker, gap=1, flat=True):
    oldinps = None
    # only work with output part of input tuples (TODO this may need adaptation for sanity check)
    if len(inputs[0])==2: 
        oldinps = inputs
        inputs = [inputs[i][1] for i in range(len(inputs))]
    finlist = []
    toklists = [toker(inp).input_ids for inp in inputs]
    for toks in toklists: 
        tmp = []
        for ind in range(gap, len(toks), gap): 
            tmp.append(toker.decode(toks[:ind+1], skip_special_tokens=True))
        if (len(toks)%gap)!=0:
            tmp.append(toker.decode(toks, skip_special_tokens=True))
        finlist.append(tmp)
    if oldinps: 
        # we want something we can use directly
        if flat: 
            rlens = [len(finlist[i]) for i in range(len(finlist))]
            inps = []
            outs = []
            for i in range(len(oldinps)): 
                inps.extend([oldinps[i][0]]*rlens[i])
                outs.extend(finlist[i])
            rlens = [sum(rlens[:i+1]) for i in range(len(rlens))]
            return [0]+rlens, [(inps[i], outs[i]) for i in range(len(outs))]
        # get the corrected reprocessed thing
        return [(oldinps[i][0], finlist[i]) for i in range(len(finlist))]
    return finlist

In [4]:
def pp_hh(ex): 
    ex['chosen'] = ex['chosen'].replace("\n\n", " ###")
    ex['rejected'] = ex['rejected'].replace("\n\n", " ###")
    return ex

def pp_uf(ex):
    ex['chosen'] = "###Human: "+ex['question'].strip()+" ###Assistant: "+ex['response_j']
    ex['rejected'] = "###Human: "+ex['question'].strip()+" ###Assistant: "+ex['response_k']
    return ex

In [5]:
# Code for loading in data
hh_train = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base", split="test").map(pp_hh, num_proc=10)

Found cached dataset json (/home/prasann/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-37c6f75e35564d2a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Loading cached processed dataset at /home/prasann/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-37c6f75e35564d2a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-a0c4ef1f168e4156_*_of_00010.arrow


In [6]:
ultrafeediff = Dataset.load_from_disk("../../data/ultrafeeddiff/").shuffle(seed=0).select(range(1000)).filter(lambda ex: ex['tokj']<150).map(pp_uf)

Loading cached shuffled indices for dataset at /data/users/prasann/Projects/rlhf-length-biases/data/ultrafeeddiff/cache-3c77a68b828b795c.arrow
Loading cached processed dataset at /data/users/prasann/Projects/rlhf-length-biases/data/ultrafeeddiff/cache-78f11653849498af.arrow
Loading cached processed dataset at /data/users/prasann/Projects/rlhf-length-biases/data/ultrafeeddiff/cache-79fd4cb5fb1100dc.arrow


In [ ]:
ultrafeediff['question']

In [30]:
rewardmodels = ["OpenAssistant/reward-model-deberta-v3-large-v2", "weqweasdas/hh_rlhf_rm_open_llama_3b", "openbmb/UltraRM-13b"]
rmstr = rewardmodels[0]
# TODO maybe we should use the same tokenizer across all of these? Aligning between models is gonna be a bit weird
tokenizer = AutoTokenizer.from_pretrained(rewardmodels[0])
model = load_rm(rmstr)

In [31]:
def scoretexts(texts, mod, gap=5): 
    test_inps = [x.rsplit("###Assistant: ", 1) for x in texts]
    print(test_inps[0])
    values, data = get_token_inps(test_inps, tokenizer, gap)
    scos = model(data)
    nscos = [scos[values[i-1]:values[i]] for i in range(1, len(values))]
    return nscos

In [36]:
with torch.no_grad():
    chosenscos = scoretexts(ultrafeediff['chosen'][:50], model, 5)
    rejscos = scoretexts(ultrafeediff['rejected'][:50], model, 5)

['###Human: In this task you will be given a list of integers. You should remove all of the integers that are divisible by 3 from the list. If every integer in the input list is divisible by 3 then an empty list should be returned. Zero is divisible by 3.\nQ: [61, 35, -86, -38, 58, -9, 78]\nA: ', 'The filtered list is: [61, 35, -86, -38, 58, -9]']


  0%|                                                                                                                                     | 0/450 [00:00<?, ?it/s]

Human: In this task you will be given a list of integers. You should remove all of the integers that are divisible by 3 from the list. If every integer in the input list is divisible by 3 then an empty list should be returned. Zero is divisible by 3.
Q: [61, 35, -86, -38, 58, -9, 78]
A:

Assistant:  The filtered list is:


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 450/450 [00:23<00:00, 19.17it/s]


['###Human: In this task you will be given a list of integers. You should remove all of the integers that are divisible by 3 from the list. If every integer in the input list is divisible by 3 then an empty list should be returned. Zero is divisible by 3.\nQ: [61, 35, -86, -38, 58, -9, 78]\nA: ', '[35, -86, -38, 58, -9, 78]\nConfidence: 95%']


  0%|▌                                                                                                                            | 4/991 [00:00<00:28, 34.99it/s]

Human: In this task you will be given a list of integers. You should remove all of the integers that are divisible by 3 from the list. If every integer in the input list is divisible by 3 then an empty list should be returned. Zero is divisible by 3.
Q: [61, 35, -86, -38, 58, -9, 78]
A:

Assistant:  [35, -86


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 991/991 [01:15<00:00, 13.21it/s]


In [37]:
def fin_accs(chos, rejs):
    rlast = [r[-1] for r in rejs]
    clast = [c[-1] for c in chos]
    return sum([clast[i]>rlast[i] for i in range(len(rejs))])

In [38]:
fin_accs(chosenscos, rejscos)

tensor([31])

In [13]:
sum(len(chosenscos[i])>len(rejscos[i]) for i in range(len(rejscos)))

17

In [39]:
chosenscos = [[float(f) for f in chose] for chose in chosenscos]
rejscos = [[float(f) for f in chose] for chose in rejscos]

with open('deberta_openassist_ufdata.pkl', 'wb') as file:
    pickle.dump((chosenscos, rejscos), file)
# with open('oasst_deberta_hhscores.pkl', 'wb') as file:
#     pickle.dump((chosenscos, rejscos), file)

In [11]:
# with open('llama3b_hhscores.pkl', 'rb') as file:
#     chosen, rejs = pickle.load((chosenscos, rejscos), file)

In [ ]:
chosenscos

In [32]:
with open('llama3b_hhscores.pkl', 'rb') as file:
    l3bchos, l3rejs = pickle.load(file)

In [34]:
with open('ultra13b.pkl', 'rb') as file:
    ultrachos, ultrarejs = pickle.load(file)

In [41]:
fin_accs(l3bchos, l3rejs)

13